In [1]:
import pandas as pd
import numpy as np

In [2]:
import pandas as pd
import numpy as np

from queries.getTestsStarts import ret as problemsStarts
from queries.getSubmissionsTime import ret as problemsEnds
from queries.getSubTimeForOpenAssess import ret as problemsAssessEnds
from queries.getVideoStartAndEnd import ret as videoStartsAndEnds
from queries.getSeqEvent import ret as seqEvents
from queries.getTextsInteractions import ret as textsInteractions
from queries.getHintsTime import ret as hintsTime 

from getThemeStartTimeForUser import getThemeStarts

In [3]:
problemsStartsDF=pd.DataFrame(problemsStarts())
problemsStartsDF['action'] = problemsStartsDF['problem_type']+' '+problemsStartsDF['problem_id']
problemsStartsDF=problemsStartsDF.loc[:,['course_id','time','user_id','action']]

In [4]:
problemsEndsDF=pd.DataFrame(problemsEnds())
problemsEndsDF['action'] = 'end ungraded '+problemsEndsDF['problem_id']
problemsEndsDF=problemsEndsDF.loc[:,['course_id','time','user_id','action']]

In [5]:
problemsAssessEndsDF=pd.DataFrame(problemsAssessEnds())
problemsAssessEndsDF['action'] = 'end graded '+problemsAssessEndsDF['problem_id']
problemsAssessEndsDF=problemsAssessEndsDF.loc[:,['course_id','time','user_id','action']]

In [6]:
videoStartsAndEndsDF=pd.DataFrame(videoStartsAndEnds())
videoStartsAndEndsDF=videoStartsAndEndsDF.loc[:,['course_id','time','user_id']]
videoStartsAndEndsDF['action'] = 'video'

In [7]:
textsInteractionsDF=pd.DataFrame(textsInteractions())
textsInteractionsDF['course_id']=videoStartsAndEndsDF['course_id'][0]
textsInteractionsDF['action']='text'

In [8]:
getThemeStartsDF=pd.DataFrame(getThemeStarts())
getThemeStartsDF=getThemeStartsDF.loc[:,['course_id','time','user_id']]
getThemeStartsDF['action']='theme start'

In [9]:
hintsTimeDF=pd.DataFrame(hintsTime())
hintsTimeDF['action']='hint'

In [10]:
seqTimeDF=pd.DataFrame(seqEvents())

In [11]:
DF=pd.concat([problemsStartsDF, problemsEndsDF,problemsAssessEndsDF,videoStartsAndEndsDF,textsInteractionsDF,getThemeStartsDF,hintsTimeDF,seqTimeDF],ignore_index=True,sort=False)

In [12]:
DF.to_csv(path_or_buf='users_sequence.csv',index=False)

In [13]:
#DF = pd.read_csv('users_sequence.csv', sep=',', encoding='utf-8')

In [14]:
users=np.unique(DF['user_id'])

In [15]:
studyModelsDF=[]
problemModelsDF=[]

In [81]:
for user in users:
    userActions=DF.loc[DF['user_id']==user].sort_values(by='time')
    sequence=userActions['action'].values
    sequenceForModel=[]
    themeArr=[]
    
    gradedPosEnd=[]
    gradedPosStart=[]
    gradedSequences=[]
    
    ungradedPosEnd=[]
    ungradedPosStart=[]
    ungradedSequences=[]
    
    studySequence=[]
    
    for step in sequence:
        if step == 'theme start':
            if len(themeArr)>1:
                sequenceForModel.append(themeArr)
            themeArr=['theme start']
        elif len(themeArr)>=1:
            if step!=themeArr[-1] or step.find('graded'):
                themeArr.append(step)
    sequenceForModel.append(themeArr)
    for actionsArr in sequenceForModel:
        if 'theme start' in actionsArr:
            actionsArr.remove('theme start')
            
        gradedPosEnd=[elem for elem in actionsArr if(elem.find('end graded')!=-1)]   
        gradedPosEnd=np.unique(gradedPosEnd)        
        #gradedPosStart=[elem  for elem in actionsArr if(elem.find('graded')!=-1 and (elem.find('ungraded')==-1))]   
        

        ungradedPosEnd=[elem  for elem in actionsArr if(elem.find('end ungraded')!=-1)]  
        ungradedPosEnd=np.unique(ungradedPosEnd)
        #ungradedPosStart=[elem  for elem in actionsArr if(elem.find('ungraded')!=-1)]   
        
        if len(gradedPosEnd)>0:            
            for end in gradedPosEnd:

                endPosInSeq=[i for i, elem in enumerate(actionsArr) if elem == end]

                startPosInSeq=[i for i, elem in enumerate(actionsArr) if (elem == (end.split('end ')[-1]) or elem == end)]
                startPosInSeq.sort()

                for endIndex in endPosInSeq:
                    if endIndex!=0:
                        startIndex=startPosInSeq[startPosInSeq.index(endIndex)-1]
                    else:
                        startIndex=endIndex
                gradedSequences.append(actionsArr[startIndex:(endIndex+1)]) 
                del actionsArr[startIndex:endIndex]     
            
        if len(ungradedPosEnd)>0:
            for end in ungradedPosEnd:

                endPosInSeq=[i for i, elem in enumerate(actionsArr) if elem == end]

                startPosInSeq=[i for i, elem in enumerate(actionsArr) if (elem == end.split('end ')[-1] or elem == end)]
                startPosInSeq.sort()

                for endIndex in endPosInSeq:
                    if endIndex!=0:
                        startIndex=startPosInSeq[startPosInSeq.index(endIndex)-1]
                    else:
                        startIndex=endIndex
                ungradedSequences.append(actionsArr[startIndex:(endIndex+1)])    
                del actionsArr[startIndex:endIndex]  
            
        for elem in actionsArr:
            if elem.find('graded')!=-1 and action.find('end')==-1:
                while elem in actionsArr:
                    actionsArr.remove(elem)
                
        while 'seq' in actionsArr:
            actionsArr.remove('seq')  
            
        while 'seq' in gradedSequences:
            gradedSequences.remove('seq')  
            
        while [] in gradedSequences:
            gradedSequences.remove([])  
        while 'seq' in ungradedSequences:
            ungradedSequences.remove('seq')  
        while [] in ungradedSequences:
            ungradedSequences.remove([])              
            
    while [] in sequenceForModel:
        sequenceForModel.remove([])

    studySeq=[]
    probSeq=[]
    for array in sequenceForModel:
        array.insert(0,'TS')
        array.append('TE')
        studySeq.extend(array)
    for array in gradedSequences:
        array.insert(0,'PS')
        array.append('PE')
        probSeq.extend(array)
    for array in ungradedSequences:
        array.insert(0,'PS')
        array.append('PE')
        probSeq.extend(array)
        
    probSeq=['CH' if x.find('end')!=-1 else x for x in probSeq]        
        
    studyStatesCounter=pd.DataFrame(columns=['TS','video','text','problem','TE'])
    studyStatesCounter['TS']=[studySeq.count('TS')]
    studyStatesCounter['TE']=[studySeq.count('TE')]
    studyStatesCounter['video']=[studySeq.count('video')]
    studyStatesCounter['text']=[studySeq.count('text')]
    studyStatesCounter['problem']=[studySeq.count('end graded')+studySeq.count('end ungraded')]
    
    study=pd.DataFrame(columns=['states','TS','video','text','problem','TE'])
    study['states']=['TS','video','text','problem','TE']
    study=study.set_index('states')
    study=study.fillna(0)
    study['TE']['TE']=1
    
    oldState=False
    
    while 'hint' in studySeq:
        studySeq.remove('hint')
        
    for state in studySeq:
        if state.find('end')!=-1:
            state='problem'
        if state.find('grade')!=-1:
            continue
        if state!='TS' and oldState!='TE' and oldState!=False:
            study[state][oldState]+=1
        oldState=state    
    
    study=study.transpose()

    for col in study.columns:
        if studyStatesCounter[col][0]:
            study[col]/=studyStatesCounter[col][0]
    study=study.transpose()
    
    study['TS']='*'    
    study=study.transpose()
    study['TE']='*'
    study=study.transpose()
    study['TE']['TS']='*'    
    
    study=study.values
    studyModel=[]
    for array in study:
        studyModel.extend(array)
    while '*' in studyModel:
        studyModel.remove('*')    
        
    studyModelsDF.append(studyModel)
    
    problemStatesCounter=pd.DataFrame(columns=['PS','video','text','hint','PV','CH','PE'])
    problemStatesCounter['PS']=[probSeq.count('PS')]
    problemStatesCounter['PE']=[probSeq.count('PS')]
    problemStatesCounter['video']=[probSeq.count('video')]
    problemStatesCounter['text']=[probSeq.count('text')]
    problemStatesCounter['hint']=[probSeq.count('hint')]
    problemStatesCounter['CH']=[probSeq.count('CH')]
    problemStatesCounter['prob view']=[probSeq.count('graded')+probSeq.count('ungraded')-probSeq.count('PS')] ######осторожно

    problem=pd.DataFrame(columns=['states','PS','video','text','hint','prob view','CH','PE'])
    problem['states']=['PS','video','text','hint','prob view','CH','PE']
    problem=problem.set_index('states')
    problem=problem.fillna(0)
    problem['PE']['PE']=1
    
    oldState=False
    while 'seq' in probSeq:
        probSeq.remove('seq')
        
    for state in probSeq:
        if state.find('graded')!=-1:
            if oldState=='PS' or state.find('end')!=-1:
                continue
            else:
                state='prob view'
            if state.find('end')!=-1:
                #обработать долю отеветов 
        if state!='PS' and oldState!='PE' and oldState!=False:
            problem[state][oldState]+=1

        oldState=state

        
    problem=problem.transpose()
    for col in problem.columns:
        if problemStatesCounter[col][0]:
            problem[col]/=problemStatesCounter[col][0]
    problem=problem.transpose()
    
    problem['PS']='*'
    tmp=problem['PE']['CH']
    problem['PE']='*'
    problem['PE']['CH']=tmp
    
    problem=problem.transpose()
    problem['PE']='*'
    problem=problem.transpose()
    
    problem=problem.values
    problemModel=[]
    for array in problem:
        problemModel.extend(array)
    while '*' in problemModel:
        problemModel.remove('*')
    weight=problemStatesCounter['video'][0]+problemStatesCounter['text'][0]+problemStatesCounter['hint'][0]
    problemModel.append(problemStatesCounter['video'][0]/weight)
    problemModel.append(problemStatesCounter['text'][0]/weight)
    problemModel.append(problemStatesCounter['hint'][0]/weight)

    problemModelsDF.append(problemModel)
    
studyModelsDF=pd.DataFrame(studyModelsDF,columns=['TS_V','TS_T','TS_P',
                                                  'V_V','V_T','V_P','V_TE',
                                                  'T_V','T_T','T_P','T_TE',
                                                  'P_V','P_T','P_P','P_TE'])
problemModelsDF=pd.DataFrame(problemModelsDF,columns=['PS_V','PS_T','PS_H','PS_PV','PS_CH',
                                                      'V_V','V_T','V_H','V_PV','V_CH',
                                                      'T_V','T_T','T_H','T_PV','T_CH',
                                                      'H_V','H_T','H_H','H_PV','H_CH',
                                                      'PV_V','PV_T','PV_H','PV_PV','PV_CH',
                                                      'CH_V','CH_T','CH_H','CH_PV','CH_CH','CH_PE',
                                                      'vid_part','text_part','hint_part'     
                                                     ])    


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [62]:
actionsArr=['end ungraded 1', '0', '1','end ungraded 1','end ungraded 1','5']
ungradedPosEnd=[elem  for elem in actionsArr if(elem.find('end ungraded')!=-1)]  

In [64]:
ungradedPosEnd

['end ungraded 1', 'end ungraded 1', 'end ungraded 1']